In [1]:
import pandas as pd 
import numpy as np

In [ ]:
class network:
    def __init__(self):
        self.network = pd.DataFrame(columns=['node1', 'node2'])
        self.nodeInfo = pd.DataFrame(columns=['nodeId','p','c'])                       
    
    def add_node (self, info):
        if info[0] in self.nodeInfo.nodeId:
            print (f'could not add node {info[0]}, node allready in network')
            return
        self.nodeInfo.loc[len(self.nodeInfo)] = info
    def add_connection(self, connection):
        for i in connection:
            if i not in self.nodeInfo['nodeId'].to_list():
                print(f'could not add connection, you need to add node {i} first')
                return
        self.network.loc[len(self.network)] = connection

    def update_node_c (self,node:int,c:int):
        ## ändert die farbe eines einzelnen angegebnenen nodes
        ## node = die id die des Nodes der geändert werden soll
        ## c = die farbe zu welcher der node geändert werden soll als int
        index = self.nodeInfo.query(f'nodeId =={node}').index[0]
        self.nodeInfo.at[index,'c'] = c

    def return_network(self):
        return self.network
    
    def find_all_conection_of_node(self, node):
        filterNode1 = self.network.query(f'node1 == {node}').node2
        filterNode2 = self.network.query(f'node2 == {node}').node1
        combinNodes = list(set(filterNode1.to_list()+filterNode2.to_list()))
        return combinNodes
    
    def check_for_path(self, node):
        allPossibleConections = []
        testedNodes =[]
        allPossibleConections = self.find_all_conection_of_node(node)
        for connectionNode in allPossibleConections:
            nextConnections = self.find_all_conection_of_node(connectionNode)
            for nc in nextConnections:
                if nc not in allPossibleConections:
                    allPossibleConections.append(nc)            
        return allPossibleConections
    
    def test_if_c_is_1(self,node):
        result = self.nodeInfo.query(f'nodeId =={node}').c 
        if not result.empty and result.iloc[0]==1:
            return True
        else:
            return False
    
    def get_mean_c_of_node(self, node):
        nodeList = self.find_all_conection_of_node(node)
        cList = []
        for node in nodeList:
            cList.append(self.nodeInfo.query(f'nodeId == {node}').c.to_numpy()[0])
        meanOfC = np.mean(cList)
        #print(f'node {node} has mean {meanOfC}')
        return(meanOfC)
    
    
    def get_p_of_node(self,node):
        result = self.nodeInfo.loc[self.nodeInfo['nodeId'] == node, 'p']
        if result.empty:
                raise ValueError(f"No matching node found for nodeId: {node}")
            
        return result.iloc[0]
        #return self.nodeInfo.query(f'nodeId == {node}').p.to_numpy()[0]
    
    def check_cascade(self):
        ## ist noch broken, muss noch öffters wiedreholt werden
        exitWhile = False
        while exitWhile == False:
            print('checking chascade ')
            nodesToCheck = self.nodeInfo.query('c == 1').nodeId.to_list()
            for singleNodeToCheck in nodesToCheck:
                connectionsOfNode = self.find_all_conection_of_node(singleNodeToCheck)
                for singleConnectionNode in connectionsOfNode:
                    ## if node allready c=1
                    if self.test_if_c_is_1(singleConnectionNode) == True:
                        break
                    pOfNode = self.get_p_of_node(singleConnectionNode)
                    meanCOfConections = self.get_mean_c_of_node(singleConnectionNode)
                    if meanCOfConections>pOfNode:
                        self.update_node_c(singleConnectionNode,1)
            nodesToCompare = self.nodeInfo.query('c == 1').nodeId.to_list()    
            if nodesToCheck == nodesToCompare:
                exitWhile = True

                    


                    
                


            




In [12]:

n = network()
n.add_node([1,0.1,0])
n.add_node([2,0.9,1])
n.add_node([3,0.9,0])
n.add_node([4,0.9,0])
n.add_node([5,0.9,0])
n.add_node([6,0.9,0])

n.add_node([9,0.5,0])
n.add_node([8,0.5,0])
n.add_connection([1,2])
n.add_connection([1,3])
n.add_connection([4,2])
n.add_connection([3,2])
n.add_connection([5,2])
n.add_connection([5,3])
n.add_connection([5,6])

n.add_connection([9,8])

test = n.return_network()
n.nodeInfo

,nodeId,p,c
0,1.0,0.1,0.0
1,2.0,0.9,1.0
2,3.0,0.9,0.0
3,4.0,0.9,0.0
4,5.0,0.9,0.0
5,6.0,0.9,0.0
6,9.0,0.5,0.0
7,8.0,0.5,0.0


In [13]:
n.test_if_c_is_1(2)

True

In [14]:
n.check_cascade()
n.nodeInfo

checking chascade 
checking chascade 


,nodeId,p,c
0,1.0,0.1,1.0
1,2.0,0.9,1.0
2,3.0,0.9,0.0
3,4.0,0.9,1.0
4,5.0,0.9,0.0
5,6.0,0.9,0.0
6,9.0,0.5,0.0
7,8.0,0.5,0.0
